#Binomial DeAmericanization Methodology


In [1]:
#As defined in the paper by Burkovska

## Calibration Function

In [2]:
#Heston Model Calibration
!pip install QuantLib-Python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 41.0 MB/s eta 0:00:00


In [5]:
import QuantLib as ql
import scipy.optimize as opt
from scipy.optimize import minimize
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

In [6]:
#Heston Model Calibration
def heston_price(params, spot, strike, maturity, rf, q, option_type="call"):
    v0, kappa, theta, sigma, rho = params
    todays_date = ql.Date().todaysDate()
    ql.Settings.instance().evaluationDate = todays_date

    #Heston Process Variables
    spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot))
    rf_curve = ql.YieldTermStructureHandle(
        ql.FlatForward(todays_date, rf, ql.Actual365Fixed())
    )
    div_curve = ql.YieldTermStructureHandle(
        ql.FlatForward(todays_date, q, ql.Actual365Fixed())
    )

    #Heston Process Definition
    heston_process = ql.HestonProcess(
        rf_curve, div_curve, spot_handle,
        v0, kappa, theta, sigma, rho
    )

    #European Option Pricing
    model = ql.HestonModel(heston_process)
    engine = ql.AnalyticHestonEngine(model)
    payoff = ql.PlainVanillaPayoff(
        ql.Option.Call if option_type == "call" else ql.Option.Put,
        strike
    )
    exercise = ql.EuropeanExercise(todays_date + int(365 * maturity))
    option = ql.VanillaOption(payoff, exercise)
    option.setPricingEngine(engine)

    return option.NPV()

#Heston Model Optimization Function
def build_static_ql_objects(spot, rf, q):
    today = ql.Date().todaysDate()
    ql.Settings.instance().evaluationDate = today

    #Heston Process Variables
    spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot))
    rf_curve = ql.YieldTermStructureHandle(
        ql.FlatForward(today, rf, ql.Actual365Fixed()))
    div_curve = ql.YieldTermStructureHandle(
        ql.FlatForward(today, q, ql.Actual365Fixed()))

    return today, spot_handle, rf_curve, div_curve

#Options Pricing Helper Function
def prebuild_option_list(market_data, today):
    option_list = []
    for opt in market_data:
        K = opt["strike"]
        T = opt["maturity"]

        payoff = ql.PlainVanillaPayoff(ql.Option.Call, K)
        exercise = ql.EuropeanExercise(today + int(T * 365))
        option = ql.VanillaOption(payoff, exercise)

        option_list.append(option)

    return option_list

#Optimized Helper Calibration Function
def fast_calibration_objective(params, market_data, option_list, spot_handle, rf_curve, div_curve):
    v0, kappa, theta, sigma, rho = params

    heston_process = ql.HestonProcess(
        rf_curve, div_curve, spot_handle,
        v0, kappa, theta, sigma, rho
    )
    model = ql.HestonModel(heston_process)
    engine = ql.AnalyticHestonEngine(model)

    errors = []
    for opt, mkt in zip(option_list, market_data):
        opt.setPricingEngine(engine)
        model_price = opt.NPV()
        errors.append((model_price - mkt["price"]) ** 2)

    return np.sum(errors)

#Main Calibration Function
def calibrate_heston(market_data, spot, rf, q):
    today, spot_handle, rf_curve, div_curve = build_static_ql_objects(spot, rf, q)
    option_list = prebuild_option_list(market_data, today)

    initial_guess = np.array([0.04, 1.0, 0.04, 0.3, -0.5])
    bounds = [
        (1e-4, 2.0),
        (1e-4, 5.0),
        (1e-4, 2.0),
        (1e-4, 5.0),
        (-0.999, 0.999)
    ]

    result = minimize(
        fast_calibration_objective,
        initial_guess,
        args=(market_data, option_list, spot_handle, rf_curve, div_curve),
        bounds=bounds,
        method="L-BFGS-B"
    )

    return result.x, result.fun

def main():
    #Synthetic Data Generation Parameters
    true_params = np.array([0.05, 1.2, 0.04, 0.4, -0.6])
    spot = 100
    rf = 0.02
    q = 0.00

    # Generate 50 × 50 synthetic grid
    strikes = np.linspace(50, 150, 50)
    maturities = np.linspace(0.1, 2.0, 50)

    #Generate Synthetic Data
    t0 = time.time()
    market_data = []
    for K in strikes:
        for T in maturities:
            price = heston_price(true_params, spot, K, T, rf, q)
            market_data.append({"strike": K, "maturity": T, "price": price})

    t1 = time.time()
    print(f"Synthetic data generated in {t1 - t0:.3f} seconds")
    #Calibrate Model
    t2 = time.time()
    calibrated_params, err = calibrate_heston(market_data, spot, rf, q)
    t3 = time.time()

    #Print Results
    print(f"Calibration completed in {t3 - t2:.3f} seconds\n")
    print("True Parameters:", true_params)
    print("Calibrated Parameters:\n")
    print("v0:", calibrated_params[0],"\nkappa:", calibrated_params[1], "\ntheta: ", calibrated_params[2], "\nsigma: ", calibrated_params[3], "\nrho:", calibrated_params[4])
    print("\nCalibration Error:", err)

if __name__ == "__main__":
    main()


Synthetic data generated in 3.953 seconds
Calibration completed in 55.731 seconds

True Parameters: [ 0.05  1.2   0.04  0.4  -0.6 ]
Calibrated Parameters:

v0: 0.04999995181804884 
kappa: 1.1999912201342686 
theta:  0.03999998501916755 
sigma:  0.3999982747623925 
rho: -0.6000001815108529

Calibration Error: 7.907881133064633e-10
